# Base agents

> To be written.

In [ ]:
#| default_exp agents.base

In [ ]:
import logging
logging_level = logging.DEBUG

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export


from abc import ABC, abstractmethod
from typing import Union
import numpy as np
from sklearn.utils.validation import check_array

from ddopnew.envs.base import BaseEnvironment

# # TEMPORARY
# from sklearn.utils.validation import check_array
# import numbers

In [ ]:
#| export

class BaseAgent():

    train_mode = "direct_fit" # or "epochs_fit" or "env_interaction"
    
    def __init__(self, environment_info):
        self.environment_info = environment_info
        self.mode = "train"

    @abstractmethod
    def draw_action(self, observation):
        pass

    def train(self):
        self.mode = "train"
        
    def eval(self):
        self.mode = "eval"

        

In [ ]:
#| export
class BaseSAAagent(BaseAgent):
    def __init__(self, environment_info):
        super().__init__(environment_info)

    def find_weighted_quantiles(self, weights, weightPosIndices, sl, y):
        
        """
        Find the weighted quantile of a range of data y. 
        It assumes that all arrays are of shape (n_samples, n_outputs).

        This function is designed for single-output only
        """

        # test shapes have lenght 2 with error
        assert len(y.shape) == 2, "y should be of shape (n_samples, n_outputs)"

        n_outputs = y.shape[1]

        yWeightPos = y[weightPosIndices]
        
        q = []

        if len(weights.shape) == 1:
            weights = weights.reshape(-1, 1)
        
        for i in range(n_outputs):
            
            indicesYSort = np.argsort(yWeightPos[:, i])

            ySorted = yWeightPos[indicesYSort, i]
            
            distributionFunction = np.cumsum(weights[indicesYSort, i])

            decisionIndex = np.where(distributionFunction >= sl)[0][0]
            
            q.append(ySorted[decisionIndex])
        
        return q
    
    def _validate_X_predict(self, X):
        """Validate X whenever one tries to predict"""

        X = check_array(X)

        n_features = X.shape[1]
        if self.n_features_ != n_features:
            raise ValueError("Number of features of the model must match the input. "
                             "Model n_features is %s and input n_features is %s "
                             % (self.n_features_, n_features))
        return X

In [ ]:
# #| export

# def check_cu_co(cu, co, n_outputs):
#     """Validate under- and overage costs.

#     Parameters
#     ----------
#     cu : {ndarray, Number or None}, shape (n_outputs,)
#        The underage costs per unit. Passing cu=None will output an array of ones.
#     co : {ndarray, Number or None}, shape (n_outputs,)
#        The overage costs per unit. Passing co=None will output an array of ones.
#     n_outputs : int
#        The number of outputs.
#     Returns
#     -------
#     cu : ndarray, shape (n_outputs,)
#        Validated underage costs. It is guaranteed to be "C" contiguous.
#     co : ndarray, shape (n_outputs,)
#        Validated overage costs. It is guaranteed to be "C" contiguous.
#     """
#     costs = [[cu, "cu"], [co, "co"]]
#     costs_validated = []
#     for c in costs:
#         if c[0] is None:
#             cost = np.ones(n_outputs, dtype=np.float64)
#         elif isinstance(c[0], numbers.Number):
#             cost = np.full(n_outputs, c[0], dtype=np.float64)
#         else:
#             cost = check_array(
#                 c[0], accept_sparse=False, ensure_2d=False, dtype=np.float64,
#                 order="C"
#             )
#             if cost.ndim != 1:
#                 raise ValueError(c[1], "must be 1D array or scalar")

#             if cost.shape != (n_outputs,):
#                 raise ValueError("{}.shape == {}, expected {}!"
#                                  .format(c[1], cost.shape, (n_outputs,)))
#         costs_validated.append(cost)
#     cu = costs_validated[0]
#     co = costs_validated[1]
#     return cu, co

# class NewsvendorSAAagentOLD(BaseAgent):

#     def __init__(self, environment_info, cu, co):
#         self.cu = cu
#         self.co = co

#         self.sl = cu / (cu + co)

#         self.quantiles = np.array([0.0])

#         super().__init__(environment_info)

#         self.fitted = False

#     def _calc_weights(self):
#         weights = np.full(self.n_samples_, 1 / self.n_samples_)
#         return weights

#     def fit(self, Y, X, mask=None):

#         demand = Y
#         features = X

#         self.mask=mask
#         y=demand

#         if mask is not None:
#             if demand.shape != mask.shape:
#                 if demand.shape[1]==1 & len(mask.shape)==1:
#                     mask = mask.reshape((-1,1))
#                     self.mask = mask
#                 if demand.shape != mask.shape:
#                     raise ValueError("Shapes of mask and demand do not match")
#                 # check if 1 either in mask.shape or demand.shape, if yes squeeze
#             demand=demand*mask
       
#         y = check_array(y, ensure_2d=False, accept_sparse='csr')

#         if y.ndim == 1:
#             y = np.reshape(y, (-1, 1))

#         # Training data
#         self.y_ = y
#         self.n_samples_ = y.shape[0]

#         # Determine output settings
#         self.n_outputs_ = y.shape[1]

#         # Check and format under- and overage costs
#         self.cu_, self.co_ = check_cu_co(self.cu, self.co, self.n_outputs_)

#         self.q_star = np.array(self._findQ(self._calc_weights()))

#         self.fitted=True

#         return self

#     def _findQ(self, weights):
#         """Calculate the optimal order quantity q"""

#         y = self.y_
#         q = []

#         for k in range(self.n_outputs_):
#             alpha = self.cu_[k] / (self.cu_[k] + self.co_[k])
#             y_product = y[:,k]
#             if self.mask is not None:
#                 mask_product = self.mask[:,k]
#                 y_product = y_product[mask_product.astype(bool)]
#             # print(y_product.shape)
#             q.append(np.quantile(y_product, alpha, interpolation="higher"))
#         print("found optimal q:", q)
#         return q

#     def draw_action(self, *args, **kwargs):

#         if self.fitted:
#             pred = self.q_star
            
#         else:
#             pred = np.random.rand(1)  
#         return pred

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()